# Basalt DatasetSDK Demo

This notebook demonstrates the asynchronous functionality of the DatasetSDK in the Basalt Python SDK.

In [ ]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  # Needed to make notebook work in VSCode

os.environ["BASALT_BUILD"] = "development"

from basalt import Basalt

# Initialize the SDK
basalt = Basalt(
    api_key="sk-f50...",  # Replace with your API key
    log_level="debug"  # Optional: Set log level
)

## Example 1: List All Datasets

This example demonstrates how to list all datasets asynchronously.

In [ ]:
async def list_datasets():
    err, datasets = await basalt.datasets.list()
    if err:
        pass
    else:
        for _dataset in datasets:
            pass
    return datasets

# Run the async function
datasets = await list_datasets()

## Example 2: Get a Specific Dataset

This example demonstrates how to retrieve a specific dataset by its slug.

In [ ]:
async def get_dataset(datasets):
    if len(datasets) > 0:
        sample_dataset = datasets[0]
        err, dataset = await basalt.datasets.get(sample_dataset.slug)
        if err:
            pass
        else:
            pass
        return sample_dataset, dataset
    else:
        return None, None

# Run the async function
sample_dataset, dataset = await get_dataset(datasets)

## Example 3: Add a Row to a Dataset

This example demonstrates how to add a new row to an existing dataset.

In [ ]:
async def add_row(sample_dataset):
    if sample_dataset:
        # Create some sample values for the dataset row
        values = {column: f"Sample {column} value" for column in sample_dataset.columns}

        err, row, warning = await basalt.datasets.add_row(
            slug=sample_dataset.slug,
            values=values,
            name="Async Sample Row",
            ideal_output="Expected output for this row",
            metadata={"source": "async_example", "type": "demo"}
        )

        if err:
            pass
        elif warning:
            pass
        else:
            pass

# Run the async function
await add_row(sample_dataset)